In [9]:
from __future__ import division
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from nltk.stem import WordNetLemmatizer
from collections import Counter

#
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

#define matplotlib styles
matplotlib.style.use('fivethirtyeight')
%matplotlib inline

In [13]:
data = pd.read_csv('train_modified.csv')
#test = pd.read_csv('test_modified.csv')

#data = pd.concat([train,test], ignore_index=True)

number_classes = (len(pd.Series.unique(data['cuisine'])))
lbl_enc = LabelEncoder()

data.head()

,cuisine,id,ingredients_string
0,greek,10259,romaine lettuce black olives grape tomatoes ga...
1,southern_us,25693,plain flour ground pepper salt tomato ground b...
2,filipino,20130,egg pepper salt mayonaise cooking oil green ch...
3,indian,22213,water vegetable oil wheat salt
4,indian,13162,black pepper shallot cornflour cayenne pepper ...


In [14]:
corpustr = data['ingredients_string']
vectorizertr = TfidfVectorizer(stop_words='english', ngram_range = ( 1, 1),analyzer="word", 
                               max_df = .6 , binary=False , token_pattern=r'\w+' , sublinear_tf=False, norm = 'l2')



In [15]:
count_matrix = vectorizertr.fit_transform(corpustr)
count_matrix.shape

(39774, 3018)

In [16]:
targets = data['cuisine']
targets = lbl_enc.fit_transform(targets)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(count_matrix ,targets ,test_size = 0.33, random_state = 42)

In [18]:
from sklearn.svm import LinearSVC
clf_SVM = LinearSVC()

start = time.time()
clf_SVM.fit(x_train,y_train)
end = time.time()

In [19]:
print "Time for Training: {}".format(end - start)
print "SVM Train score: %.4f" % clf_SVM.score(x_train,y_train)
print "SVM Test score: %.4f" % clf_SVM.score(x_test,y_test)

Time for Training: 1.48001599312
SVM Train score: 0.8670
SVM Test score: 0.7854


In [9]:
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV   

clf_xgb = XGBClassifier(learning_rate =0.1,
                        n_estimators=1000,
                        max_depth=3,
                        min_child_weight=1,
                        gamma=0,
                        subsample=0.8,
                        colsample_bytree=0.8,
                        objective= 'multi:softmax',
                        nthread = 4,
                        scale_pos_weight=1,
                        seed=27)
start = time.time()
clf_xgb.fit(x_train,y_train, eval_metric='merror')
end = time.time()

/Users/lucas/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
prediction_train = clf_xgb.predict(x_train)
prediction_test = clf_xgb.predict(x_test)

In [11]:
print "Time for Training: {}".format(end - start)
print "XGboost Train score: %.4f" %(metrics.accuracy_score(y_train,prediction_train))
print "XGboost Test score: %.4f" %(metrics.accuracy_score(y_test,prediction_test))

Time for Training: 516.27013588
XGboost Train score: 0.9533
XGboost Test score: 0.7887


In [13]:
from sklearn.naive_bayes import MultinomialNB

clf_NB = MultinomialNB()

start = time.time()
clf_NB.fit(x_train.toarray(),y_train)
end = time.time()

print "Time for Training: {}".format(end - start)
print "NB Train score: %.4f" % clf_NB.score(x_train.toarray(),y_train)
print "NB Test score: %.4f" % clf_NB.score(x_test.toarray(),y_test)

Time for Training: 0.654016971588
NB Train score: 0.6771
NB Test score: 0.6598


In [14]:
from sklearn import linear_model

clf_SGD = linear_model.SGDClassifier()

start = time.time()
clf_SGD.fit(x_train.toarray(),y_train)
end = time.time()

print "Time for Training: {}".format(end - start)
print "SGD Train score: %.4f" % clf_SGD.score(x_train.toarray(),y_train)
print "SGD Test score: %.4f" % clf_SGD.score(x_test.toarray(),y_test)

Time for Training: 10.4978721142
SGD Train score: 0.7983
SGD Test score: 0.7608


In [15]:
from sklearn.neural_network import MLPClassifier

clf_MLP = MLPClassifier(solver = 'lbfgs', alpha = 1e-2, hidden_layer_sizes=(60,50), random_state=42)

start = time.time()
clf_MLP.fit(x_train.toarray(),y_train)
end = time.time()

print "Time for Training: {}".format(end - start)
print "SGD Train score: %.4f" % clf_MLP.score(x_train.toarray(),y_train)
print "SGD Test score: %.4f" % clf_MLP.score(x_test.toarray(),y_test)

Time for Training: 140.771771908
SGD Train score: 0.8687
SGD Test score: 0.7643


In [16]:
from sklearn.ensemble import RandomForestClassifier

clf_RF = RandomForestClassifier()

start = time.time()
clf_RF.fit(x_train.toarray(),y_train)
end = time.time()

print "Time for Training: {}".format(end - start)
print "SGD Train score: %.4f" % clf_RF.score(x_train.toarray(),y_train)
print "SGD Test score: %.4f" % clf_RF.score(x_test.toarray(),y_test)

Time for Training: 7.72605681419
SGD Train score: 0.9929
SGD Test score: 0.6826


In [28]:
param_grid = [{'C':[1,10,100,100]}]

clf_GS = GridSearchCV(clf_SVM,param_grid=param_grid)

clf_GS.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'C': [1, 10, 100, 100]}], pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [30]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf_SVC,x_train,y_train,cv = 5)

In [31]:
scores

array([ 0.77848576,  0.77873617,  0.7727955 ,  0.7805107 ,  0.77268458])